In [1]:
# Importando bibliotecas necessárias:

from pandas_datareader import data
import datetime as dt
import numpy as np
import pandas as pd

import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory


In [2]:
END_DATE = str(dt.date.today())

#### Calculando parâmetros para o modelo:

In [288]:
# Tickers das ações:

A = ['ABEV3.SA','ASAI3.SA','CCRO3.SA','BOVA11.SA','ITUB4.SA','LREN3.SA','RENT3.SA','PRIO3.SA','SLCE3.SA','VBBR3.SA',
    'WALM34.SA','U1BE34.SA','NFLX34.SA','BCSA34.SA','AIRB34.SA'] # Carteira nova futura + carteira BB BDR

p = list()

# Obtendo preços atuais de cada ação:

for ticker in A:
    
    price = data.DataReader(ticker, 'yahoo', start=END_DATE, end=END_DATE)['Close'][0]
    
    p.append(price)

In [289]:
dict(zip(A,p))

{'ABEV3.SA': 16.18000030517578,
 'ASAI3.SA': 18.420000076293945,
 'CCRO3.SA': 13.180000305175781,
 'BOVA11.SA': 111.70999908447266,
 'ITUB4.SA': 29.6200008392334,
 'LREN3.SA': 30,
 'RENT3.SA': 64.55000305175781,
 'PRIO3.SA': 28.75,
 'SLCE3.SA': 43.369998931884766,
 'VBBR3.SA': 18.649999618530273,
 'WALM34.SA': 42.810001373291016,
 'U1BE34.SA': 34.47999954223633,
 'NFLX34.SA': 24.809999465942383,
 'BCSA34.SA': 12.609999656677246,
 'AIRB34.SA': 27.15999984741211}

In [322]:
# Porcentagens de ações BR e de BDR:

cent_BR = [0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1] # porcentagens ações nacionais

cent_BDR = [(1)/5]*5 # porcentagens ações internacionais

cent = [percentage/2 for percentage in (cent_BR+cent_BDR)] # todas as porcentagens divididas por 2

tol = 0.008  # tolerância percentual para exceder o limite na carteira (parâmetro ajustável para dar factibilidade ao modelo)

cent_min = [percentage-tol for percentage in cent] # porcentagens mínimas

cent_max = [percentage+tol for percentage in cent] # porcentagens máximas

C = 5200 # capital máximo a ser investido

In [323]:
# Modelo:

model = pyo.ConcreteModel()

#### Set de ações

In [324]:
# Conjunto de ações

model.set_A = pyo.Set(initialize = list(range(len(A))))

#### Variável de decisão

In [325]:
# Criando uma variável X inteira que diz respeito à quantidade comprada de cada ação:

model.x = pyo.Var(model.set_A, within=NonNegativeIntegers)

x = model.x

## Modelo


#### Função objetivo

## $$ max \displaystyle \sum_{a \in A}p_{a}x_{a}$$

In [326]:
# Função objetivo do modelo

model.exprobj = pyo.Expression()

model.exprobj = sum(p[a]*x[a] for a in model.set_A)

model.obj = pyo.Objective(expr= model.exprobj, sense = pyo.maximize)


#### Restrições

#### I)

## $$ \displaystyle \sum_{a \in A}p_{a}x_{a} \leq C$$

    A restrição (I) força que a quantidade investida não supere a quantidade de capital disponível.

In [327]:
# Restrição (1):

model.C1 = pyo.ConstraintList()

model.C1.add(expr= sum(p[a]*x[a] for a in model.set_A) <= C)

#### II), III)

## $$ \frac{p_{a}x_{a}}{C} \leq Cent_{a max}, \forall a $$
## $$ \frac{p_{a}x_{a}}{C} \geq Cent_{a min}, \forall a$$

    As restrições (II) e (III) forçam, respectivamente, que a quantidade da ação "a" na carteira não supere uma porcentagem máxima definida, mas estejam acima de uma porcentagem mínima definida.

In [328]:
model.C2 = pyo.ConstraintList()

model.C3 = pyo.ConstraintList()

for a in model.set_A:
    
    model.C2.add(p[a]*x[a]/C <= cent_max[a])

    model.C3.add(p[a]*x[a]/C >= cent_min[a])

In [329]:
opt = pyo.SolverFactory('glpk', executable='C:\glpk-4.65\w64\glpsol')

In [330]:
results = opt.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\User\AppData\Local\Temp\tmpo_61jds5.glpk.raw --wglp C:\Users\User\AppData\Local\Temp\tmpu_hwe09o.glpk.glp
 --cpxlp C:\Users\User\AppData\Local\Temp\tmp2pxvinvz.pyomo.lp
Reading problem data from 'C:\Users\User\AppData\Local\Temp\tmp2pxvinvz.pyomo.lp'...
32 rows, 16 columns, 46 non-zeros
15 integer variables, none of which are binary
196 lines were read
Writing problem data to 'C:\Users\User\AppData\Local\Temp\tmpu_hwe09o.glpk.glp'...
160 lines were written
GLPK Integer Optimizer, v4.65
32 rows, 16 columns, 46 non-zeros
15 integer variables, none of which are binary
Preprocessing...
1 row, 12 columns, 12 non-zeros
12 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.261e+01  max|aij| =  4.281e+01  ratio =  3.395e+00
GM: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
EQ: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
2N: min|aij| =  6.

In [349]:
# Printando solução

lista_sol = []

for a in model.set_A:
    
    dict_sol = dict()
    
    dict_sol['name'] = 'X_' + A[a][:-3]
    
    dict_sol['value'] = pyo.value(x[a])
    
    lista_sol.append(dict_sol)
    
    print(A[a], pyo.value(x[a]))
    
df_sol = pd.DataFrame(lista_sol)

ABEV3.SA 18.0
ASAI3.SA 16.0
CCRO3.SA 19.0
BOVA11.SA 2.0
ITUB4.SA 10.0
LREN3.SA 10.0
RENT3.SA 4.0
PRIO3.SA 8.0
SLCE3.SA 6.0
VBBR3.SA 13.0
WALM34.SA 13.0
U1BE34.SA 14.0
NFLX34.SA 22.0
BCSA34.SA 38.0
AIRB34.SA 18.0


In [350]:
# Calculando demais dados do dataframe

df_sol['price'] = p

df_sol['qtd_investida'] = df_sol['value']*df_sol['price']

df_sol['porcentagem_teorica'] = [100*round(p,4) for p in cent]

df_sol['porcentagem_real'] = round(df_sol['qtd_investida']/C, 4)*100

df_sol['difenca_porcentagem'] = df_sol['porcentagem_teorica'] - df_sol['porcentagem_real']

In [351]:
nome_arquivo = 'sol-' + END_DATE + '.csv' # criando nome para o novo arquivo

df_sol.to_csv(nome_arquivo, index=False) # exportando novo arquivo

In [352]:
print(nome_arquivo)

sol-2022-10-03.csv


In [353]:
df_sol

,name,value,price,qtd_investida,porcentagem_teorica,porcentagem_real,difenca_porcentagem
0,X_ABEV3,18.0,16.180000,291.240005,5.0,5.60,-0.60
1,X_ASAI3,16.0,18.420000,294.720001,5.0,5.67,-0.67
2,X_CCRO3,19.0,13.180000,250.420006,5.0,4.82,0.18
3,X_BOVA11,2.0,111.709999,223.419998,5.0,4.30,0.70
4,X_ITUB4,10.0,29.620001,296.200008,5.0,5.70,-0.70
5,X_LREN3,10.0,30.000000,300.000000,5.0,5.77,-0.77
6,X_RENT3,4.0,64.550003,258.200012,5.0,4.97,0.03
7,X_PRIO3,8.0,28.750000,230.000000,5.0,4.42,0.58
8,X_SLCE3,6.0,43.369999,260.219994,5.0,5.00,0.00
9,X_VBBR3,13.0,18.650000,242.449995,5.0,4.66,0.34


#### Aqui, o programa irá ler um arquivo .csv com as ações da carteira e irá dizer quantas e quais ações comprar ou vender.

In [354]:
nome_arquivo_antigo = 'sol-2022-04-04.csv' # aqui vai o nome do arquivo correspondente à última atualização

df_past = pd.read_csv(nome_arquivo_antigo)

In [355]:
df_past

,name,value,price,qtd_investida,porcentagem_teorica,porcentagem_real,difenca_porcentagem
0,X_AMER3,9.0,33.410000,300.689999,5.00,5.01,-0.01
1,X_BBDC4,11.0,21.809999,239.909994,3.75,4.00,-0.25
2,X_BRKM5,7.0,44.400002,310.800011,5.50,5.18,0.32
3,X_BRML3,32.0,9.700000,310.399994,5.00,5.17,-0.17
4,X_CSNA3,13.0,26.230000,340.989994,5.50,5.68,-0.18
5,X_JHSF3,49.0,6.760000,331.240011,5.00,5.52,-0.52
6,X_MRFG3,17.0,21.270000,361.590008,6.00,6.03,-0.03
7,X_PCAR3,13.0,25.080000,326.039999,5.50,5.43,0.07
8,X_SLCE3,4.0,48.930000,195.720001,3.75,3.26,0.49
9,X_VALE3,3.0,97.949997,293.849991,5.00,4.90,0.10


In [356]:
# Tratando as ações em comum, ações novas e ações antigas

# A ideia é dar como output a quantidade de ações a serem vendidas e compradas

acoes_comuns = np.intersect1d(df_sol['name'], df_past['name'])

df_sol_intersect = df_sol.query('name in @acoes_comuns') # ações comuns

df_past_intersect = df_past.query('name in @acoes_comuns') # ações comuns

df_sol_not_intersect = df_sol.query('name not in @acoes_comuns') # ações compradas

df_past_not_intersect = df_past.query('name not in @acoes_comuns') # ações vendidas

In [357]:

# Mostrando quantidade de ações comuns a serem compradas/vendidas:

print('Ações comuns: \n')

for variavel in acoes_comuns:
    
    qtd_nova = int(df_sol_intersect[df_sol_intersect['name'] == variavel]['value'])
    
    qtd_antiga = int(df_past_intersect[df_past_intersect['name'] == variavel]['value'])
    
    var_qtd = qtd_nova - qtd_antiga
    
    if var_qtd < 0: # se a variação é negativa, então é para venda
        
        print('Vender {} ações de {}.'.format(abs(var_qtd), variavel[2:]))
        
    if var_qtd > 0: # se a variação é positiva, então é para compra
        
        print('Comprar {} ações de {}.'.format(abs(var_qtd), variavel[2:]))

                    # se a variação é igual a 0, mantem-se a quantidade

# Mostrando quantidade de ações novas a serem compradas:

print('\nAções novas: \n')

for ticker in range(len(df_sol_not_intersect)):
    
    print('Comprar {} ações de {}'.format(int(df_sol_not_intersect.iloc[ticker]['value']), df_sol_not_intersect.iloc[ticker]['name'][2:]))

    
# Mostrando quantidade de ações antigas a serem vendidas:

print('\nAções antigas: \n')

for ticker in range(len(df_past_not_intersect)):
    
    print('Vender {} ações de {}'.format(int(df_past_not_intersect.iloc[ticker]['value']), df_past_not_intersect.iloc[ticker]['name'][2:]))


Ações comuns: 

Comprar 2 ações de SLCE3.
Comprar 2 ações de WALM34.

Ações novas: 

Comprar 18 ações de ABEV3
Comprar 16 ações de ASAI3
Comprar 19 ações de CCRO3
Comprar 2 ações de BOVA11
Comprar 10 ações de ITUB4
Comprar 10 ações de LREN3
Comprar 4 ações de RENT3
Comprar 8 ações de PRIO3
Comprar 13 ações de VBBR3
Comprar 14 ações de U1BE34
Comprar 22 ações de NFLX34
Comprar 38 ações de BCSA34
Comprar 18 ações de AIRB34

Ações antigas: 

Vender 9 ações de AMER3
Vender 11 ações de BBDC4
Vender 7 ações de BRKM5
Vender 32 ações de BRML3
Vender 13 ações de CSNA3
Vender 49 ações de JHSF3
Vender 17 ações de MRFG3
Vender 13 ações de PCAR3
Vender 3 ações de VALE3
Vender 8 ações de CATP34
Vender 11 ações de COCA34
Vender 8 ações de COWC34
Vender 18 ações de NVDC34
Vender 7 ações de UNHH34
